## My Classifier test score
- Binary Classifier for MNIST: 92.89%
- Multi Class Classifier for MNIST: 93.56%
- Multi Class Classifier for CIFAR10: 51.47%

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

c:\Users\jwhau\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Exercise 1-3
class MLP_CIFAR10(nn.Module) :
    def __init__(self, in_dim, out_dim, device, inter_dim=16*16):
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True, device=device)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True, device=device)
        self.act2 = nn.ReLU()
        self.linear3 = nn.Linear(inter_dim, inter_dim, bias=True, device=device)
        self.act3 = nn.ReLU()
        self.linear4 = nn.Linear(inter_dim, out_dim, bias=True, device=device)
        
    def forward(self, x) :
        # assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        x = x.reshape(-1, self.in_dim)
        z = self.linear1(x)
        z = self.act1(z)
        z = self.linear2(z)
        z = self.act2(z)
        z = self.linear3(z)
        z = self.act3(z)
        z = self.linear4(z)
        return z

In [3]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

In [4]:
from torchvision.datasets import CIFAR10

# download path 정의
download_root = './CIFAR10_DATASET'

train_dataset = CIFAR10(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = CIFAR10(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = CIFAR10(download_root, transform=mnist_transform, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# option 값 정의
batch_size = 32
dim = 3*32*32
learning_rate = 1e-1

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

STUDENT_model = MLP_CIFAR10(dim, 10, device)

In [7]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)

Number of Parameters:  920842


In [8]:
my_loss = nn.CrossEntropyLoss()

In [9]:
for i, (data, target) in enumerate(train_loader):
    print(data.shape, target.shape)
    if i==0:
        break

torch.Size([32, 3, 32, 32]) torch.Size([32])


In [10]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(5):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data.to(device))
        
        loss = my_loss(output, target.to(device))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 1.7587270023574146
Epoch: 1 | Train Loss: 1.4734369853102696
Epoch: 2 | Train Loss: 1.3466686971738258
Epoch: 3 | Train Loss: 1.2529741300273536
Epoch: 4 | Train Loss: 1.1723549000666222


In [11]:
''' Test for train_dataset '''

#image_size = 28*28
image_size = 3*32*32 # for CIFAR dataset
STUDENT_model.eval()
batch_size = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size).to(device)),dim=1)
    count += (prediction == label.to(device)).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size).to(device)),dim=1)
    count += (prediction == label.to(device)).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for train_dataset: correctly matched/total: 30237/50000 accuracy: 60.47%

Test for test_dataset: correctly matched/total: 5245/10000 accuracy: 52.45%

